In [37]:
import selenium
import pandas as pd
import csv
import requests
import html5lib
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

PATH = Service("C:\Program Files (x86)\chromedriver.exe")

In [59]:
# Initializes driver, opens web page, creates csv file/reader
driver = webdriver.Chrome(service=PATH)
driver.get("https://www.binghamton-ny.gov/home")

csv_file = open("job_openings.csv", "w", newline="", encoding="utf-8")
csv_writer = csv.writer(csv_file)
csv_writer.writerow(["job title", "job_type", "application deadline", "salary"])

# Defines tabs used in action chain
government_tab = driver.find_element(By.XPATH, "//*[@id='dropdownrootitem3']")
departments_tab = driver.find_element(By.XPATH, "//*[@id='dropdownrootitem3']/div/div/ul[1]/li/a")

# Creates action chain and 
actions = ActionChains(driver)
actions.move_to_element(government_tab)
actions.click(government_tab)
actions.click(departments_tab)
actions.perform()

try:
    # finds personnel link, clicks on it    
    personnel_link = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, "//*[@id='widget_4_33_127']/ul/li[16]/a"))
    )
    personnel_link.click()
    
    # finds employment link, clicks on it
    employment_link = WebDriverWait(driver, 10).until(
    EC.presence_of_element_located((By.XPATH, "//*[@id='widget_1040_107_145']/p[1]/a"))
    )
    employment_link.click()
    
    # Switches driver to the new tab
    driver.switch_to.window(driver.window_handles[1])
    
    # finds table and iterates over the rows
    table = driver.find_element(By.XPATH, "//*[@id='ColumnUserControl4']/div[2]/table")
    rows = table.find_elements(By.XPATH, ".//tr")
    for row in rows:
        # gets all items as list of the columns, formatted as follows:
        # job title
        # job type
        # application deadline
        # salary
        cols = row.find_elements(By.XPATH, ".//td")
        
        # uses while loop to parse the data we need and writes to csv
        i = 0
        while i < len(cols):
            job_title = cols[i].text
            job_type = cols[i+1].text
            application_deadline = cols[i+2].text
            salary = cols[i+3].text
            i += 4
            
            csv_writer.writerow([job_title, job_type, application_deadline, salary])

finally:
    driver.quit()
    csv_file.close()

In [57]:
sheet = pd.read_csv("job_openings.csv")
sheet

,job title,job_type,application deadline,salary
0,Animal Control Officer,Full Time,Open Until Filled,"$45,228.00 per Year"
1,Assistant Engineer,Full Time,Open Until Filled,"$38,127 - $45,062 per Year"
2,Building Inspector II,Full Time,Open Until Filled,"$42,436 per Year"
3,City Engineer,Full Time,Open Until Filled,Not Specified
4,Code Inspector,Full Time,Open Until Filled,"$33,632 per Year*"
5,Commissioner of Public Works,Full Time,Open Until Filled,Not Specified
6,Crossing Guard,Part Time,Open Until Filled,$43.75 per Day
7,Director of Police Department Community Outrea...,Full Time,Open Until Filled,"$66,950.00 per Year"
8,First Deputy Commissioner of Public Works,Full Time,Open Until Filled,"$61,730.00 per Year"
9,Laboratory Technician (Sanitary Services),Full Time,Open Until Filled,"$29,406.00 per Year"
